## Data Profiling


The reasoning behind having a data profiling is twofold: first, it aims at assessing the impact that unknown datasets may have on your MDM system. At the beginning of any data-related project, you usually don't know how exactly the data from your customer looks like (and neither does the customer). Getting to know the data and its inherent quality is always crucial in an MDM project, as the supplier of an MDM system is always at risk of loading data that is not according to defined standards or simply, in a real bad shape. Failing to assess data quality at an early stage might as well render the purpose of an MDM system useless (garbage in, garbage out). On top of that, matching & deduplication software is usually sensitive to certain triggers that may appear in the data; some matching components conduct analytical steps for themselves, i.e. to identify tokens that contain legal forms in the data. It is important to know how such tokens interact with the matching software to prevent unwanted side effects. 

Secondly, it is also about communicating these findings to the customer. Usually, the customer is not aware of the condition of his data. By providing a concise and reproducible examination of his data, the customer will start to build confidence in your skills and learn to perceive you as a trusted advisor in all data-related matters. Raising awareness for the costs of poorly managed data may also spark a change in his mindset and open up possibilities for future data governance initiatives.

The present data profiling toolkit aims at combining the shared intentions of customers and data analysts; they both seek to explore and understand data alike. Thus, this script investigates the inherent data structure to predicate the impact of the findings on the overall validation and matching outcome. It has been developed and tested to suit German customer data in particular. A fair share of German customer data still revolves around name and address entities. Thus, it pays special attention to particularities of the German language, namely compound words. I.e. the German counterpart to "legal department" is "Rechtsabteilung", a concatenated word. Compound words pose a bit of a challenge when it comes to token analysis. Let's stay with this example and assume "department", "Abteilung" in German - is a keyword used in the analysis step of the matching component. In fact, "Abteilung" will in most cases be part of a larger compound word rather than a standalone, which makes it harder to detect. In order to discover such occurrences, the script will follow a few steps:
    
1. load data into R
2. convert everything to lower case and replace special characters in data with a blank (you would be surprised how often you'd find "+", "/" as a name separator, especially in legacy systems)
3. split words on whitespace into single tokens
4. prepare list A containing all tokens of character length 2 and more (to include abbrev. & roman numerals)
5. prepare data frame B including all tokens that pass a certain threshold (token frequency and character length)
6. store list A in elasticsearch 
7. query elasticsearch with tokens from data frame B to detect keywords hidden in compound words and collect examples for each search query (works just like a grep)
8. assign percentages to each token relatively to its overall occurrence
9. display results in sortable table

A few supporting technologies have been added to the bundle: elasticsearch was chosen over a classical grep, simply because it is more handy to search or add some nice visualizations somewhere down the line. A more prominent role is given to [Jupyter Notebooks](http://jupyter.org/). The Jupyter Notebook is a powerful application that allows creating and sharing documents that contain live code, visualizations and explanatory text with customers and peers. [Docker Container](http://www.docker.com/) provide a great means of transporting the required infrastructure (Anaconda distribution, Jupyter Notebooks, R and elasticsearch). 

<br>


Load required R libraries.

In [1]:
require(readr)
require(plyr)
require(elastic)
require(httr)
require(DT)

Loading required package: readr
Warning message:
"package 'readr' was built under R version 3.2.5"Loading required package: plyr
Warning message:
"package 'plyr' was built under R version 3.2.2"Loading required package: elastic
Warning message:
"package 'elastic' was built under R version 3.2.5"
Attaching package: 'elastic'

Das folgende Objekt ist maskiert 'package:plyr':

    count

Loading required package: httr
Warning message:
"package 'httr' was built under R version 3.2.5"Loading required package: DT
Warning message:
"package 'DT' was built under R version 3.2.5"

Choose file (expected format is csv). Mind that some data ingestion packages convert characters to factors during load. Select proper encoding. You might also want to have a look at the structure of the data using str().

In [10]:
raw_data <- read_delim(file.choose(), delim = ";", col_names = TRUE, locale = locale(encoding = "UTF-8"))
#str(raw_data)

Warning message:
"134466 parsing failures.
row              col   expected   actual
 57 waz_mo_timefrom1 valid date 08:00:00
 57 waz_mo_timeto1   valid date 16:00:00
 57 waz_di_timefrom1 valid date 08:00:00
 57 waz_di_timeto1   valid date 16:00:00
 57 waz_mi_timefrom1 valid date 08:00:00
... ................ .......... ........
.See problems(...) for more details."

Remove special characters and split words into tokens. Make a preselection on token length and relative frequency.

In [3]:
name <- unlist(lapply(raw_data$name1, strsplit, split = " ")) # name1 is a placeholder for the name column, please change manually
name <- tolower(name)
name <- lapply(name, gsub, pattern = "[[:punct:]]", replacement = " ")
name <- unlist(lapply(name, strsplit, split = " "))
name <- name[order(nchar(name), decreasing = TRUE)]
name <- name[nchar(name) >= 2]
name_token_count <- plyr::count(name)
name_token_count$token_len <- unlist(lapply(as.vector(name_token_count$x), nchar))
name_token_count <- subset(name_token_count, freq >= 5 & token_len >= 3 | freq >= 50)
name_token_count <- arrange(name_token_count, desc(freq))

Load tokens into elastic (unfortunately, the docs_bulk()-loader doesn't work properly atm).

In [4]:
connect()
if(!index_exists("bamboo-shack")){
  index_create("bamboo-shack")
}

transport:  http 
host:       127.0.0.1 
port:       9200 
path:       NULL 
username:   NULL 
password:   <secret> 
errors:     simple 
headers (names):  NULL 

In [ ]:
name_tokens_ls <- setNames(as.list(name), rep_len(c("name"), length(name)))
j <- 1
for(j in 1:length(name)){
  load_silently <- POST(url = "http://localhost:9200/bamboo-shack/bamboo-shack",  body = name_tokens_ls[j], content_type("application/json"), encode = "json")
}

In [5]:
if(elastic::count(index = "bamboo-shack") == length(name)){
  print("All name tokens have been loaded successfully to elasticsearch.")
} else{ print("Warning: Data loaded to elasticsearch doesn't match the amount of name tokens.") }


[1] "All name tokens have been loaded successfully to elasticsearch."


Query elasticsearch with the pre-selected tokens to detect the famous German word concatenations. Collect both total amount of findings as well as two examples for each search query.


In [6]:
examples <- vector(mode = "list", length = length(name_token_count$x))
req_prep_list <- laply(name_token_count$x, sprintf, fmt = '*%s*')
total_hits <- vector(mode = "list", length = length(name_token_count$x))
for(q in 1:length(req_prep_list)){
  response <- Search(index = "bamboo-shack", q = req_prep_list[q], size = 10000)
  total_hits[q] <- response$hits$total
  examples[q] <- paste(as.character(response$hits$hits[[1]]$`_source`$name), 
                                  as.character(response$hits$hits[[3]]$`_source`$name),
                                  sep = " - ")
}

One can compare the amount of words in the response of elastic with the relative frequency of tokens. If the total hits in elastic exceed the token frequency, this means the token reappears in other words as well and does not only match with itself.

In [7]:
name_token_count$total_hits <- unlist(total_hits)
name_token_count$examples <- unlist(examples)

Order data according to multiple-hits and display acumulative percentages.

In [8]:
name_token_count <- arrange(name_token_count, desc(total_hits))
name_token_count$perc <- unlist(lapply(name_token_count$total_hits, function(X) round((X/sum(name_token_count$total_hits))*100, digits = 3)))
name_token_count$acc_perc <- cumsum(name_token_count$perc)
name_token_count <- name_token_count[c("x", "examples", "freq", "total_hits", "perc", "acc_perc")]
names(name_token_count) <- c("Token", "Examples in data", "Frequency of the token itself", "Overall frequency (including as part of other words)",  
                             "relative percentage", "accumulated percentage")

Display findings in layouted, sortable table.

In [9]:
datatable(name_token_count, options = list(pageLength = 10))

Warning message in instance$preRenderHook(instance):
"It seems your data is too big for client-side DataTables. You may consider server-side processing: http://rstudio.github.io/DT/server.html"

HTML widgets cannot be represented in plain text (need html)